<a href="https://colab.research.google.com/github/fjtm/Data/blob/master/dogsVScats_francisco_javier_tebar_martinez.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Comentarios profesor:

Utilizar accuracy. 

Parecido a Random Forest.

Primer bloque tratamiento de los datos. Cargar y tratar las imágenes (resize...), cada imagen tiene un tamaño diferente. 
Escalar las imágnes, montar el tensor, normalizar a (-0.5, 0.5). No pintar la versión normalizada. 

Random suffle para mezclar perros y gatos. 

Modelo básico. 

Conseguir almenos un 60% de accuracy

Para mejorar el modelo (opcional).

Earlystopping (recomendable).

Gridsearch (sin gpu no hacerlo). 

Tecnicas de aumentación de datos (recomendable).


Al final un informe, diciendo lo que hemos probado y que da...


Empezar con imágnes pequeñas y luego ir subiendo. 

# Lab assignment: dogs VS cats

<table><tr>
    <td><img src="img/doge.jpg" style="width:400px;height:400px;"></td>
    <td><img src="img/cat.jpg" style="width:400px;height:400px;"></td>
</tr></table>

In this assignment we will face an image classification problem, trying to tell apart images of dogs and images of cats. The final battle begins! And to do so we will use a Deep Learning approach.

## Guidelines

Throughout this notebook you will find empty cells that you will need to fill with your own code. Follow the instructions in the notebook and pay special attention to the following symbols.

<img src="img/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
You will need to solve a question by writing your own code or answer in the cell immediately below or in a different file, as instructed.</font>

***

<img src="img/exclamation.png" height="80" width="80" style="float: right;"/>

***
<font color=#2655ad>
This is a hint or useful observation that can help you solve this assignment. You should pay attention to these hints to better understand the assignment.
</font>

***

<img src="img/pro.png" height="80" width="80" style="float: right;"/>

***
<font color=#259b4c>
This is an advanced exercise that can help you gain a deeper knowledge into the topic. Good luck!</font>

***

To avoid missing packages and compatibility issues you should run this notebook under one of the [recommended Deep Learning environment files](https://github.com/albarji/teaching-environments/tree/master/deeplearning).

The following code will embed any plots into the notebook instead of generating a new window:

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline

Lastly, if you need any help on the usage of a Python function you can place the writing cursor over its name and press Caps+Shift to produce a pop-out with related documentation. This will only work inside code cells. 

Let's go!

## Data processing

The images to use in this assignment are available [here](https://drive.google.com/open?id=105jGDrjEgxx2W2gYhFF8Mbf1J9qJeDPm). Download the data package and extract it into a folder in your computer.

Data is structured under two folders, *train* and *test*. You are only allowed to use the *test* data to measure the performance of your model **after** you have finished training. Both *train* and *test* folders contain one subfolder per class (*cats* and *dogs*).

The recommended way to load an image from disk is to use the *imread* function from the *scikit-image* library. For example:

In [2]:
from skimage.io import imread

image = imread("./img/sample.jpg")

FileNotFoundError: ignored

This returns the image as 3-dimensional numpy matrix, containing the value of intensity of every pixel in each row, column and color channel.

In [0]:
image.shape

We can visualize the loaded image using

In [0]:
plt.imshow(image)

The dataset includes 20000 training images and 5000 test images, each image at a different resolution. Loading all this data at once requires too much memory, so we will need to **resize** each image to a smaller, common size as we load them from disk. We can do so by again making use of a *scikit-image* function. For example:

In [0]:
from skimage.transform import resize

size = 64
smallimage = resize(image, (64, 64, 3), mode="reflect")  # 64 x 64 pixels, 3 color channels
plt.imshow(smallimage)

<img src="img/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
    
Perform the following tasks to prepare the data:
- Build a numpy array *X_train* containing one entry for each training image in the data. Each entry must contain the 3-dimensional matrix of pixels corresponding to an image. Make sure to scale down each image to a common size
- Normalize *X_train* so that pixels follow a distribution lying in the range [-0.5,0.5]
- Build a corresponding array *Y_train* with the class of each training image (0 for cats and 1 for dogs).
- Randomly shuffle the data. Make sure you shuffle both *X_train* and *Y_train* using the same permutation, so you don't lose track of the class of each training pattern.

- Repeat the whole procedure again to generate *X_test* and *Y_test* matrices for the test data.
</font>

***

<img src="img/exclamation.png" height="80" width="80" style="float: right;"/>

***
<font color=#2655ad>
- Take advantage of the *glob* package to generate the filename lists for the indoor and outdoor subfolders.
- Since you need to repeat the procedure both for training and test data, it would be useful to implement a function that performs all the data loading and transforming steps.
</font>

***

**Creamos una carpeta en el directorio actual con los datos de la práctica almacenados en github (Este proceso es un poco lento pero permite ejecutar el notebook tanto en local, como en google colab). Si bien, si no se trabaja con google colab y  se disponen de los datos, recomiendo crear una carpeta de nombre Data en el directorio actual con los datos de train y test para mayor velocidad, en vez de clonar el repositorio de github.** 

In [2]:
!git clone --recursive https://github.com/fjtm/Data.git

fatal: destination path 'Data' already exists and is not an empty directory.


In [0]:
def preprocesing(image, size = 64, mode = "reflect"):
    return(np.array(resize(imread(image), (size, size, 3), mode=mode, anti_aliasing=True)-0.5))

In [0]:
import glob
from skimage.transform import resize
from skimage.io import imread
import numpy as np
import pandas as pd

In [5]:
X_train = []
Y_train = []
X_test = []
Y_test = []

print("Loading cats train")
for image in glob.glob('Data/train/cats/*.jpg'):
    X_train.append(preprocesing(image))
    Y_train.append(0)
print("Loading dogs train")
for image in glob.glob('Data/train/dogs/*.jpg'):
    X_train.append(preprocesing(image))
    Y_train.append(1)
print("Loading cats test")
for image in glob.glob('Data/test/cats/*.jpg'):
    X_test.append(preprocesing(image))
    Y_test.append(0)
print("Loading dogs test")
for image in glob.glob('Data/test/dogs/*.jpg'):
    X_test.append(preprocesing(image))
    Y_test.append(1)
print("Complete")

Loading cats train
Loading dogs train
Loading cats test
Loading dogs test
Complete


If everything has been properly loaded both X_train and Y_train lists should have equal length, and the same should happen for the pair X_test and Y_test.

In [6]:
print("X_train size", len(X_train))
print("Y_train size", len(Y_train))
print("X_test size", len(X_test))
print("Y_test size", len(Y_test))

X_train size 20000
Y_train size 20000
X_test size 5000
Y_test size 5000


Cambiamos el formato de X_train y X_test a un ndarray con (número de imagenes, 64, 64, 3) y mezclamos las imágenes. En este trabajo no se trabajará con imagenes con mayor resulición ya que sino podemos tener problemas de memoria.

In [0]:
np.random.seed(12)
index_train = np.arange(20000)
np.random.shuffle(index_train)
index_test = np.arange(5000)
np.random.shuffle(index_test)

In [0]:
X_train = np.asarray(X_train)[index_train]
X_test = np.asarray(X_test)[index_test]
Y_train = np.asarray(Y_train)[index_train]
Y_test = np.asarray(Y_test)[index_test]

Modificamos las etiquetas creando las categorías.  

In [9]:
from keras.utils import np_utils

Using TensorFlow backend.


In [0]:
Y_train = np_utils.to_categorical(Y_train, 2)
Y_test = np_utils.to_categorical(Y_test, 2)

## Basic deep model

In [0]:
from keras.models import Sequential
from keras.layers.core import Dense
from keras.layers.core import Activation
from keras.layers.core import Dropout
from keras.callbacks import EarlyStopping
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers.core import Flatten
from keras.preprocessing.image import ImageDataGenerator

<img src="img/question.png" height="80" width="80" style="float: right;"/>

***
<font color=#ad3e26>
Create a deep network that achieves at least a 60% of accuracy in the test set. Make use of the layers you seem fit for problem. You cannot use the test data for fitting the network, but you can use it to check the final performance of different network architectures, and select the architecture performing best.</font>

***

En esta primera parte mostraré algunas de las CNN que he probado y que van aumentando en complejidad hasta la arquitectura elegida:

<ol>
1. CNN 1:
    <br>
    <ul>
        <li> 32 capas convolucionales con kernels (3,3) (Con función de activación Relu y tratamiento de bordes valid)
        <li> Capa de MaxPooling (2,2)
        <li> Capa Flatten
        <li> Capa Densa con 2 neuronas (Con función de activación sigmoide)
    </ul>  
2. CNN 2:
    <ul>
        <li> 32 capas convolucionales con kernels (3,3) (Con función de activación Relu y tratamiento de bordes valid)
        <li> Capa de MaxPooling (2,2)
        <li> Capa Flatten
        <li> Capa densa con 64 neuronas (Con función de activación Relu) y dropout (0.5)
        <li> Capa densa con 2 neuronas (Con función de activación sigmoide)
    </ul>
3. CNN 3:
    <ul>
        <li> 32 capas convolucionales con kernels (3,3) (Con función de activación Relu y tratamiento de bordes valid)
        <li> Capa de MaxPooling (2,2)
        <li> 64 capas convolucionales con kernels (3,3) (Con función de activación Relu y tratamiento de bordes valid)
        <li> Capa de MaxPooling (2,2)
        <li> Capa Flatten
        <li> Capa densa con 256 neuronas (Con función de activación Relu) y dropout (0.5)
        <li> Capa densa con 2 neuronas (Con función de activación sigmoide)
    </ul>
4. CNN 4: 
    <ul>
        <li> 32 capas convolucionales con kernels (3,3) (Con función de activación Relu y tratamiento de bordes same)
        <li> Capa de MaxPooling (2,2)
        <li> 64 capas convolucionales con kernels (3,3) (Con función de activación Relu y tratamiento de bordes same)
        <li> Capa de MaxPooling (2,2)
        <li> 128 capas convolucionales con kernels (3,3) (Con función de activación Relu y tratamiento de bordes same)
        <li> Capa de MaxPooling (2,2)
        <li> 256 capas convolucionales con kernels (3,3) (Con función de activación Relu y tratamiento de bordes same)
        <li> Capa de MaxPooling (2,2)
        <li> Capa Flatten
        <li> Capa densa con 512 neuronas (Con función de activación Relu) y dropout (0.5)
        <li> Capa densa con 2 neuronas (Con función de activación sigmoide)
    </ul>
5. CNN 5:
    <ul>
        <li> 32 capas convolucionales con kernels (3,3) (Con función de activación Relu y tratamiento de bordes same)
        <li> 32 capas convolucionales con kernels (3,3) (Con función de activación Relu y tratamiento de bordes same)
        <li> Capa de MaxPooling (2,2)
        <li> 64 capas convolucionales con kernels (3,3) (Con función de activación Relu y tratamiento de bordes same)
        <li> 64 capas convolucionales con kernels (3,3) (Con función de activación Relu y tratamiento de bordes same)
        <li> Capa de MaxPooling (2,2)
        <li> 128 capas convolucionales con kernels (3,3) (Con función de activación Relu y tratamiento de bordes same)
        <li> 128 capas convolucionales con kernels (3,3) (Con función de activación Relu y tratamiento de bordes same)
        <li> Capa de MaxPooling (2,2)
        <li> 256 capas convolucionales con kernels (3,3) (Con función de activación Relu y tratamiento de bordes same)
        <li> Capa de MaxPooling (2,2)
        <li> Capa Flatten
        <li> Capa densa con 512 neuronas (Con función de activación Relu) y dropout (0.5)
        <li> Capa densa con 2 neuronas (Con función de activación sigmoide)
    </ul>
  </ol>

Antes de utilizar estos modelos es necario tomar algunas consideraciones.
<ol>
<li> <b>Función de pérdida</b> </li>
    <ul>
De entre la gran variedad de funciones de pérdida disponibles en keras, he elegido <b>binary crossentropy</b> puesto que tenemos un problema de clasificiación con solamente dos clases. Esta se puede expresar como:

$$H_p(q) = -\frac{1}{N_0 + N_1}[\sum_{i=1}^{N_0}log(p(y_i)) + \sum_{i=1}^{N_1}log(p(y_i))]$$
con $N_0$ el número de gatos y $N_1$ el número de perros. 
</ul>
<br>
<li> <b>Solver de optimización</b> </li>
<ul> 
En este caso utilizaremos el solver de optimización <b>adam</b> ya que funciona bien para problemas con muchos datos y/o parámetros.
</ul>
<br>
<li> <b>Número de epochs</b> </li>
<ul>    
Designa el número de veces que la red neuronal utilizará todos los datos de entrenamiento para actualizar los pesos. En este caso, de manera un poco manual considero que unas <b>10</b> o <b>20</b> epochs son suficientes. 
</ul>
<br>
<li> <b>Tamaño del batch</b> </li>
<ul>   
Designa el tamaño del subconjunto de datos tomados de train para actualizar los parámetros. En este caso utilizaremos <b>128</b>, lo que equivale sobre nuestro dataset de 20000 imágenes de entrenamiento a unas 150 imágnes. 
</ul>
<br>
<li> <b>Métrica</b> </li>
<ul>
Puesto que tenemos un problema con las clases perfectamente balanceadas podemos utilizar una métrica simple como es <b>accuracy</b>. 
</ol>

## CNN 1

Creamos la arquitectura del modelo y lo compilamos con el solver anterior y la función de perdida anterior. 

In [12]:
img_rows = 64 # Tamaño de la imagen de entrada (Filas)
img_cols = 64 # Tamaño de la imagen de entrada (Columnas)
kernel_size = 3 # Tamaño del kernel de convolución
pool_size = 2 # Tamaño de la región de pulido

model = Sequential()

model.add(Convolution2D(32, (kernel_size, kernel_size), padding='valid', input_shape=(img_rows, img_cols, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(pool_size, pool_size)))

model.add(Flatten())

model.add(Dense(2,activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


Entrenamos la red con el tamaño del batch y el número de epochs

In [13]:
model.fit(
    X_train, 
    Y_train, 
    batch_size=128, 
    epochs=20, 
    verbose=2 
)

Instructions for updating:
Use tf.cast instead.
Epoch 1/20
 - 8s - loss: 0.6151 - acc: 0.6618
Epoch 2/20
 - 3s - loss: 0.5562 - acc: 0.7207
Epoch 3/20
 - 2s - loss: 0.5118 - acc: 0.7563
Epoch 4/20
 - 2s - loss: 0.4893 - acc: 0.7703
Epoch 5/20
 - 2s - loss: 0.4675 - acc: 0.7834
Epoch 6/20
 - 2s - loss: 0.4548 - acc: 0.7937
Epoch 7/20
 - 2s - loss: 0.4378 - acc: 0.7988
Epoch 8/20
 - 2s - loss: 0.4239 - acc: 0.8098
Epoch 9/20
 - 2s - loss: 0.4139 - acc: 0.8137
Epoch 10/20
 - 2s - loss: 0.4078 - acc: 0.8189
Epoch 11/20
 - 2s - loss: 0.3967 - acc: 0.8236
Epoch 12/20
 - 2s - loss: 0.3877 - acc: 0.8303
Epoch 13/20
 - 2s - loss: 0.3784 - acc: 0.8364
Epoch 14/20
 - 2s - loss: 0.3699 - acc: 0.8401
Epoch 15/20
 - 2s - loss: 0.3632 - acc: 0.8419
Epoch 16/20
 - 2s - loss: 0.3559 - acc: 0.8504
Epoch 17/20
 - 2s - loss: 0.3494 - acc: 0.8516
Epoch 18/20
 - 2s - loss: 0.3426 - acc: 0.8564
Epoch 19/20
 - 2s - loss: 0.3366 - acc: 0.8596
Epoch 20/20
 - 2s - loss: 0.3283 - acc: 0.8636


Vemos la actuación del modelo sobre los datos de test.

In [14]:
score = model.evaluate(X_test, Y_test)
print("Test loss %.4f" % score[0])
print("Test accuracy %.4f" % score[1])

5000/5000 [==============================] - 1s 117us/step
Test loss 0.4993
Test accuracy 0.7726


Con este modelo bastante simple conseguimos un accuracy mayor del 70%

## CNN 2

Estudiemos los resultados con la segunda arquitectura considerada.

In [15]:
img_rows = 64 # Tamaño de la imagen de entrada (Filas)
img_cols = 64 # Tamaño de la imagen de entrada (Columnas)
kernel_size = 3 # Tamaño del kernel de convolución
pool_size = 2 # Tamaño de la región de pulido

model = Sequential()

model.add(Convolution2D(32, (kernel_size, kernel_size), padding='valid', input_shape=(img_rows, img_cols, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(pool_size, pool_size)))

model.add(Flatten())

model.add(Dense(16))
model.add(Activation("relu"))
model.add(Dropout(0.5))

model.add(Dense(2,activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [16]:
model.fit(
    X_train, 
    Y_train, 
    batch_size=128, 
    epochs=20, 
    verbose=2 
)

Epoch 1/20
 - 3s - loss: 0.6700 - acc: 0.5593
Epoch 2/20
 - 3s - loss: 0.6374 - acc: 0.5947
Epoch 3/20
 - 3s - loss: 0.6211 - acc: 0.6044
Epoch 4/20
 - 3s - loss: 0.6088 - acc: 0.6159
Epoch 5/20
 - 3s - loss: 0.5997 - acc: 0.6178
Epoch 6/20
 - 3s - loss: 0.5897 - acc: 0.6232
Epoch 7/20
 - 3s - loss: 0.5795 - acc: 0.6285
Epoch 8/20
 - 3s - loss: 0.5700 - acc: 0.6323
Epoch 9/20
 - 3s - loss: 0.5626 - acc: 0.6369
Epoch 10/20
 - 3s - loss: 0.5608 - acc: 0.6414
Epoch 11/20
 - 3s - loss: 0.5545 - acc: 0.6459
Epoch 12/20
 - 3s - loss: 0.5503 - acc: 0.6506
Epoch 13/20
 - 3s - loss: 0.5372 - acc: 0.6549
Epoch 14/20
 - 3s - loss: 0.5362 - acc: 0.6583
Epoch 15/20
 - 3s - loss: 0.5332 - acc: 0.6547
Epoch 16/20
 - 3s - loss: 0.5276 - acc: 0.6578
Epoch 17/20
 - 3s - loss: 0.5198 - acc: 0.6601
Epoch 18/20
 - 3s - loss: 0.5135 - acc: 0.6650
Epoch 19/20
 - 3s - loss: 0.5142 - acc: 0.6604
Epoch 20/20
 - 3s - loss: 0.5074 - acc: 0.6634


In [17]:
score = model.evaluate(X_test, Y_test)
print("Test loss %.4f" % score[0])
print("Test accuracy %.4f" % score[1])

5000/5000 [==============================] - 1s 133us/step
Test loss 0.4938
Test accuracy 0.7528


Tenemos un resultado similar al del modelo anterior.

## CNN 3

In [0]:
img_rows = 64 # Tamaño de la imagen de entrada (Filas)
img_cols = 64 # Tamaño de la imagen de entrada (Columnas)
kernel_size = 3 # Tamaño del kernel de convolución
pool_size = 2 # Tamaño de la región de pulido

model = Sequential()

model.add(Convolution2D(32, (kernel_size, kernel_size), padding='valid', input_shape=(img_rows, img_cols, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(pool_size, pool_size)))

model.add(Convolution2D(64, (kernel_size, kernel_size), padding='valid'))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(pool_size, pool_size)))

model.add(Flatten())

model.add(Dense(256))
model.add(Activation("relu"))
model.add(Dropout(0.5))

model.add(Dense(2,activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [19]:
model.fit(
    X_train, 
    Y_train, 
    batch_size=128, 
    epochs=20, 
    verbose=2 
)

Epoch 1/20
 - 4s - loss: 0.6062 - acc: 0.6621
Epoch 2/20
 - 4s - loss: 0.4979 - acc: 0.7607
Epoch 3/20
 - 4s - loss: 0.4455 - acc: 0.7922
Epoch 4/20
 - 4s - loss: 0.4050 - acc: 0.8138
Epoch 5/20
 - 4s - loss: 0.3721 - acc: 0.8343
Epoch 6/20
 - 4s - loss: 0.3303 - acc: 0.8554
Epoch 7/20
 - 4s - loss: 0.2883 - acc: 0.8776
Epoch 8/20
 - 4s - loss: 0.2423 - acc: 0.9030
Epoch 9/20
 - 4s - loss: 0.2097 - acc: 0.9161
Epoch 10/20
 - 4s - loss: 0.1630 - acc: 0.9383
Epoch 11/20
 - 4s - loss: 0.1309 - acc: 0.9512
Epoch 12/20
 - 4s - loss: 0.1070 - acc: 0.9608
Epoch 13/20
 - 4s - loss: 0.0878 - acc: 0.9702
Epoch 14/20
 - 4s - loss: 0.0674 - acc: 0.9783
Epoch 15/20
 - 4s - loss: 0.0561 - acc: 0.9824
Epoch 16/20
 - 4s - loss: 0.0507 - acc: 0.9844
Epoch 17/20
 - 4s - loss: 0.0471 - acc: 0.9856
Epoch 18/20
 - 4s - loss: 0.0393 - acc: 0.9884
Epoch 19/20
 - 4s - loss: 0.0444 - acc: 0.9864
Epoch 20/20
 - 4s - loss: 0.0334 - acc: 0.9888


In [20]:
score = model.evaluate(X_test, Y_test)
print("Test loss %.4f" % score[0])
print("Test accuracy %.4f" % score[1])

5000/5000 [==============================] - 1s 145us/step
Test loss 0.7148
Test accuracy 0.8215


En este caso conseguimos superar el 80% de accuracy. Sin embargo, si vemos el accuracy durante el entrenamiento, el modelo está realizando un claro overfit. 

## CNN 4

Notese que en este caso utilizamos el enfoque *same* ya que al utilizar tantas capas de MaxPooling necesitamos preservar que el tamaño de la imagen tras pasar las capas de convolución sea razonable.  

In [0]:
img_rows = 64 # Tamaño de la imagen de entrada (Filas)
img_cols = 64 # Tamaño de la imagen de entrada (Columnas)
kernel_size = 3 # Tamaño del kernel de convolución
pool_size = 2 # Tamaño de la región de pulido

model = Sequential()

model.add(Convolution2D(32, (kernel_size, kernel_size), padding='same', input_shape=(img_rows, img_cols, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(pool_size, pool_size)))

model.add(Convolution2D(64, (kernel_size, kernel_size), padding='same'))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(pool_size, pool_size)))

model.add(Convolution2D(128, (kernel_size, kernel_size), padding='same'))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(pool_size, pool_size)))

model.add(Convolution2D(256, (kernel_size, kernel_size), padding='same'))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(pool_size, pool_size)))

model.add(Flatten())

model.add(Dense(512))
model.add(Activation("relu"))
model.add(Dropout(0.5))

model.add(Dense(2,activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [22]:
model.fit(
    X_train, 
    Y_train, 
    batch_size=128, 
    epochs=10, 
    verbose=2 
)

Epoch 1/10
 - 6s - loss: 0.6293 - acc: 0.6315
Epoch 2/10
 - 4s - loss: 0.5030 - acc: 0.7540
Epoch 3/10
 - 4s - loss: 0.4074 - acc: 0.8144
Epoch 4/10
 - 4s - loss: 0.3395 - acc: 0.8517
Epoch 5/10
 - 4s - loss: 0.2825 - acc: 0.8811
Epoch 6/10
 - 5s - loss: 0.2391 - acc: 0.8985
Epoch 7/10
 - 4s - loss: 0.1885 - acc: 0.9236
Epoch 8/10
 - 4s - loss: 0.1365 - acc: 0.9477
Epoch 9/10
 - 4s - loss: 0.0918 - acc: 0.9651
Epoch 10/10
 - 4s - loss: 0.0640 - acc: 0.9765


In [23]:
score = model.evaluate(X_test, Y_test)
print("Test loss %.4f" % score[0])
print("Test accuracy %.4f" % score[1])

5000/5000 [==============================] - 1s 177us/step
Test loss 0.4580
Test accuracy 0.8608


Mejoramos el resultado anterior pero claramente la red sigue haciendo overfitting. 

## CNN 5

In [0]:
img_rows = 64 # Tamaño de la imagen de entrada (Filas)
img_cols = 64 # Tamaño de la imagen de entrada (Columnas)
kernel_size = 3 # Tamaño del kernel de convolución
pool_size = 2 # Tamaño de la región de pulido

model = Sequential()

model.add(Convolution2D(32, (kernel_size, kernel_size), padding='same', input_shape=(img_rows, img_cols, 3)))
model.add(Activation("relu"))
model.add(Convolution2D(32, (kernel_size, kernel_size), padding='same'))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(pool_size, pool_size)))

model.add(Convolution2D(64, (kernel_size, kernel_size), padding='same'))
model.add(Activation("relu"))
model.add(Convolution2D(64, (kernel_size, kernel_size), padding='same'))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(pool_size, pool_size)))

model.add(Convolution2D(128, (kernel_size, kernel_size), padding='same'))
model.add(Activation("relu"))
model.add(Convolution2D(128, (kernel_size, kernel_size), padding='same'))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(pool_size, pool_size)))

model.add(Convolution2D(256, (kernel_size, kernel_size), padding='same'))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(pool_size, pool_size)))

model.add(Flatten())

model.add(Dense(512))
model.add(Activation("relu"))
model.add(Dropout(0.5))

model.add(Dense(2,activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [25]:
model.fit(
    X_train, 
    Y_train, 
    batch_size=128, 
    epochs=10, 
    verbose=2 
)

Epoch 1/10
 - 9s - loss: 0.6755 - acc: 0.5658
Epoch 2/10
 - 8s - loss: 0.5729 - acc: 0.7020
Epoch 3/10
 - 8s - loss: 0.4729 - acc: 0.7769
Epoch 4/10
 - 8s - loss: 0.3838 - acc: 0.8310
Epoch 5/10
 - 8s - loss: 0.3294 - acc: 0.8572
Epoch 6/10
 - 8s - loss: 0.2765 - acc: 0.8823
Epoch 7/10
 - 8s - loss: 0.2306 - acc: 0.9053
Epoch 8/10
 - 8s - loss: 0.1980 - acc: 0.9195
Epoch 9/10
 - 8s - loss: 0.1669 - acc: 0.9334
Epoch 10/10
 - 8s - loss: 0.1374 - acc: 0.9446


In [26]:
score = model.evaluate(X_test, Y_test)
print("Test loss %.4f" % score[0])
print("Test accuracy %.4f" % score[1])

5000/5000 [==============================] - 1s 232us/step
Test loss 0.3226
Test accuracy 0.8801


Obtenemos el mejor resultado hasta al momento llegando casi al 90% de acierto. En este caso aunque la red sigue haciendo overfitting este no es tan pronunciado como las CNN anteriores. Así, quedemonos con esta arquitectura. 

<img src="img/exclamation.png" height="80" width="80" style="float: right;"/>

***
<font color=#2655ad>
    
If you are unable to attain the required 60% of accuracy, or your accuracy is too close to 100%, review the following checklist:
- Have you mixed training and test data?
- Have the values of the pixels been normalized?
- Have you compiled the model with a loss function appropriate for **binary** classification?
- Have you used ReLU units in the hidden layers? Have you used sigmoid or softmax layers in the output layer?

***

## Improving the network

<img src="img/pro.png" height="80" width="80" style="float: right;"/>

***

<font color=#259b4c>
    
You can further improve your results by applying one or more of the following strategies:
- Keep apart a portion of the training data as a **validation set**. Then use an <a href="https://keras.io/callbacks/#earlystopping">**EarlyStopping strategy**</a> to monitor the loss of these validation data, and stop when training when after a number of iterations such loss has not decreased.
- Make use of **metamodelling techniques** to select the best architecture parameters for the networks. You can use scikit-learn methods GridSearchCV or RandomSearchCV. You can do those by using the <a href="https://keras.io/scikit-learn-api/">KerasClassifier</a> wrapper. Even better, you can mix KerasClassifier with an advanced search method such as [BayesSearch](https://scikit-optimize.github.io/#skopt.BayesSearchCV), included in the [scikit-optimize](https://scikit-optimize.github.io/) library.
- Use **image augmentation techniques** to artifically create new training images. To do so, you can make use of <a href="https://keras.io/preprocessing/image/">ImageDataGenerator</a> in Keras.
- Make use of a pre-trained large network, building your network on top of it. You can find some examples in the [Keras Applications](https://keras.io/applications/) page.

If you correctly use these techniques you should be able to improve your model. With some effort it is possible to up to a 90% test accuracy. Try to do your best!</font>

***

IMPORTANTE

Ejecutar esta parte del código sin google colab requiere una gran cantidad de tiempo de entrenamiento. De esta manera, he guardado el modelo creado en el entorno de google colab en github. 

Una vez seleccionada la arquitectura de la red neuronal intentaremos mejorar el rendimiento de esta mediante las técnicas sugeridas. En concreto nos basaremos principalmente en dos de ellas:

<ol>
<li> <b>EarlyStopping strategy</b> </li>
    <ul>
Permite detener el entrenamiento si la función de pérdida en el conjunto de validación no decrece durante un cierto número de epochs, en concreto utilizaremos 5 epochs como margen. Puesto que 5 iteraciones pueden llevar a un mayor ovefit sin una mejora en el modelo, utilizaremos el parámetro <b>restore_best_weights</b> el cual devuelve los mejores pesos de las últimas 5 epochs en caso de que la función de perdida no decrezca durante estas iteraciones. Esta es una estrategia muy buena para controlar el overfit en las CNN.
</ul>
<br>
<li> <b>ImageDataGenerator</b> </li>
<ul> 
Esta técnica permite modificar las imagenes de train ligeramente mediante rotaciones, translaciones, zoom... Con esto se logra aumentar cosiderablemente el número de imágenes con las que entrenas el modelo sin necesidad de disponer de imágenes nuevas. 
</ul>
<br>

Para poder llevar a cabo las técnicas anteriormente comentadas es necesario separar un porcentaje de imágenes que utilizaremos para ir validando el modelo. Así, utilizaremos 2000 imágenes para validar y las 18000 restantes junto con la tecnica de aumentación de imágenes para entrenar la red. 

In [0]:
X_validate = X_train[0:2000]
Y_validate = Y_train[0:2000]
X_not_validate = X_train[2000:len(X_train)]
Y_not_validate = Y_train[2000:len(X_train)]

Creamos el generador de imágenes.

In [0]:
datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='reflect')

datagen.fit(X_not_validate)

Definimos la arquitectura.

In [0]:
img_rows = 64 # Tamaño de la imagen de entrada (Filas)
img_cols = 64 # Tamaño de la imagen de entrada (Columnas)
kernel_size = 3 # Tamaño del kernel de convolución
pool_size = 2 # Tamaño de la región de pulido

model = Sequential()

model.add(Convolution2D(32, (kernel_size, kernel_size), padding='same', input_shape=(img_rows, img_cols, 3)))
model.add(Activation("relu"))
model.add(Convolution2D(32, (kernel_size, kernel_size), padding='same'))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(pool_size, pool_size)))

model.add(Convolution2D(64, (kernel_size, kernel_size), padding='same'))
model.add(Activation("relu"))
model.add(Convolution2D(64, (kernel_size, kernel_size), padding='same'))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(pool_size, pool_size)))

model.add(Convolution2D(128, (kernel_size, kernel_size), padding='same'))
model.add(Activation("relu"))
model.add(Convolution2D(128, (kernel_size, kernel_size), padding='same'))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(pool_size, pool_size)))

model.add(Convolution2D(256, (kernel_size, kernel_size), padding='same'))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(pool_size, pool_size)))

model.add(Flatten())

model.add(Dense(512))
model.add(Activation("relu"))
model.add(Dropout(0.5))

model.add(Dense(2,activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [30]:
# Ejecutar solamente en google colab o en un entorno con GPU

model.fit_generator(datagen.flow(X_not_validate, Y_not_validate, batch_size=128),
                    steps_per_epoch=len(X_not_validate)/128, epochs=50, verbose=2,
                    validation_data = (X_validate, Y_validate),
                    callbacks=[EarlyStopping(monitor='val_loss', mode='min', verbose=2, patience=5, restore_best_weights=True)] 
)


Epoch 1/50
 - 20s - loss: 0.6858 - acc: 0.5425 - val_loss: 0.6501 - val_acc: 0.6405
Epoch 2/50
 - 17s - loss: 0.6368 - acc: 0.6364 - val_loss: 0.5990 - val_acc: 0.6707
Epoch 3/50
 - 18s - loss: 0.5863 - acc: 0.6914 - val_loss: 0.5057 - val_acc: 0.7590
Epoch 4/50
 - 18s - loss: 0.5400 - acc: 0.7318 - val_loss: 0.4961 - val_acc: 0.7597
Epoch 5/50
 - 19s - loss: 0.5103 - acc: 0.7522 - val_loss: 0.4714 - val_acc: 0.7772
Epoch 6/50
 - 18s - loss: 0.4752 - acc: 0.7759 - val_loss: 0.3894 - val_acc: 0.8210
Epoch 7/50
 - 18s - loss: 0.4465 - acc: 0.7930 - val_loss: 0.3906 - val_acc: 0.8192
Epoch 8/50
 - 18s - loss: 0.4293 - acc: 0.8028 - val_loss: 0.3544 - val_acc: 0.8368
Epoch 9/50
 - 18s - loss: 0.4114 - acc: 0.8117 - val_loss: 0.3759 - val_acc: 0.8380
Epoch 10/50
 - 19s - loss: 0.3849 - acc: 0.8246 - val_loss: 0.3061 - val_acc: 0.8660
Epoch 11/50
 - 18s - loss: 0.3631 - acc: 0.8379 - val_loss: 0.2962 - val_acc: 0.8700
Epoch 12/50
 - 18s - loss: 0.3573 - acc: 0.8380 - val_loss: 0.2695 - val_a

# Nueva sección

In [31]:
score = model.evaluate(X_test, Y_test)
print("Test loss %.4f" % score[0])
print("Test accuracy %.4f" % score[1])

5000/5000 [==============================] - 1s 212us/step
Test loss 0.1858
Test accuracy 0.9264


## Analyzing the results

<img src="img/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
Find and show some test images for which your model fails. Can you give an explanation for this behavior?</font>

***

In [0]:
####### INSERT YOUR CODE HERE

<img src="img/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
Write a small report summarizing the network choices you have tried, what worked and what didn't. What have you learned from this experience?</font>

***

Valid, sin earlystopping ni nada.

Con capa densa de 16 neuronas


Convolucional 3 (32, 3, 3) (64, 3, 3) (128, 3, 3) un 86.1% batch_size=128


Convolucional 1 (32, 3, 3) un 77% de accuracy. Sin capa densa de 16 neuronas batch_size=128

Convolucional 2 (32, 3, 3) (32, 3, 3) un 79-80%. Sin capa densa de 16 neuronas batch_size=128

Convolucional 2 (32, 3, 3) (64, 3, 3) un 82.42%. Sin capa densa de 16 neuronas batch_size=128

Convolucional 3 (32, 3, 3) (64, 3, 3) (128, 3, 3) un 87.8% batch_size=128

Convolucional 3 (32, 3, 3) (64, 3, 3) (128, 3, 3) con una red densa 128 neuronas (90% drop out)  88.52% batch_size=128

Convolucional 3 (32, 3, 3) (64, 3, 3) (128, 3, 3) con una red densa 256 neuronas (95% drop out)  87.44% batch_size=128

Same. No da buenos resultados de momento.

